In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummaryX import summary
from UnarySim.sw.kernel.linear import UnaryLinear
from UnarySim.sw.bitstream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressiveError


In [29]:
import torchvision.models as models
alexnet = models.alexnet()

In [41]:
print(alexnet.state_dict)

<bound method Module.state_dict of AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=92

In [40]:
for param in alexnet.parameters():
    print(param.size())

torch.Size([64, 3, 11, 11])
torch.Size([64])
torch.Size([192, 64, 5, 5])
torch.Size([192])
torch.Size([384, 192, 3, 3])
torch.Size([384])
torch.Size([256, 384, 3, 3])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([4096, 9216])
torch.Size([4096])
torch.Size([4096, 4096])
torch.Size([4096])
torch.Size([1000, 4096])
torch.Size([1000])


In [45]:
# for name, module in alexnet.named_modules():
#     print(name)
dir(alexnet)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backend',
 '_backward_hooks',
 '_buffers',
 '_construct',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_tracing_name',
 '_version',
 'add_module',
 'apply',
 'avgpool',
 'buffers',
 'children',
 'classifier',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'features',
 'float',
 'forward',
 'half

In [55]:
for name, buf in alexnet.named_modules():
    print(buf)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [12]:
in_feature = 1
out_feature = 1
mode = "unipolar"
scaled = True
rng = "Sobol"
bias=False

fc = nn.Linear(in_feature, out_feature, bias=bias)
if mode is "unipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(256).round().div(256)
    if bias is True:
        fc.bias.data = torch.rand(1, out_feature).mul(256).round().div(256)
elif mode is "bipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(2).sub(1).mul(256).round().div(256)
    if bias is True:
        fc.bias.data = torch.rand(1, out_feature).mul(2).sub(1).mul(256).round().div(256)
ufc = UnaryLinear(in_feature, out_feature, 256, fc.weight, fc.bias, mode=mode, scaled=scaled, bias=bias)

print("weight:")
print(fc.weight.data)
print(fc.weight.size())
if bias is True:
    print("\nbias:")
    print(fc.bias)
    print(fc.bias.size())

iVec = (torch.rand(1, in_feature)*256).round()/256
oVec = fc(iVec)

print("\ninput:")
print(iVec)
print("\noutput:")
print(oVec)

print(iVec*fc.weight.data)
print(iVec.matmul(fc.weight.data))

if scaled is False:
    oVec = oVec.clamp(-1.,1.).mul(256).floor()/256
else:
    if bias is True:
        oVec = oVec.div(in_feature).mul(256).floor()/256
    else:
        oVec = oVec.div(in_feature).mul(256).floor()/256

print("\nquantized output:")
print(oVec)

Parameter containing:
tensor([[9]])
rng_wght_idx: tensor([[0]])
weight:
tensor([[0.0352]])
torch.Size([1, 1])

input:
tensor([[0.9570]])

output:
tensor([[0.0336]], grad_fn=<MmBackward>)
tensor([[0.0336]])
tensor([[0.0336]])

quantized output:
tensor([[0.0312]], grad_fn=<DivBackward0>)


In [16]:
iVecSource = SourceGen(iVec, bitwidth=8, mode=mode).Gen()

iVecRNG = RNG(8, 1, rng).Out()
iVecBS = BSGen(iVecSource, iVecRNG)

iVecPP = ProgressiveError(iVec, mode=mode)
oVecPP = ProgressiveError(oVec, mode=mode)

with torch.no_grad():
    idx = torch.zeros(iVecSource.size()).type(torch.long)
    for i in range(256):
        iBS = iVecBS.Gen(idx + i)
        iVecPP.Monitor(iBS)
        oVecU = ufc(iBS)

        oVecPP.Monitor(oVecU)
    print("input error: ", min(min(iVecPP.Report())), max(max(iVecPP.Report())))
    print("output error:", min(min(oVecPP.Report())), max(max(oVecPP.Report())))
    print(oVecPP.Report())
    print(oVecPP.one_cnt)
    print(oVecPP.out_pp)



input error:  tensor(0.) tensor(0.)
output error: tensor(0.1055) tensor(0.1055)
tensor([[0.1055]])
tensor([[55.]])
tensor([[0.2148]])
